In [77]:
# Notebook to save the time profile of events for reading into simulatoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
import math 

In [78]:
Mode = ["CRAB","CRAB"]

data = pd.read_csv(f"../Files/{Mode[0]}/EventInfo_merge_{Mode[1]}.csv", delimiter = ",", names = ["event", "x", "y", "z", "t"])

# Cut the first bin
data = data[data['t'] > 200]
data = data.reset_index(drop=True)
display(data)

,event,x,y,z,t
0,1,-0.175,-0.228,-10.638,13859.8
1,1,-0.176,-0.230,-10.659,13964.8
2,1,-0.173,-0.229,-10.660,13967.9
3,1,-0.172,-0.231,-10.662,13973.7
4,1,-0.174,-0.231,-10.667,13996.1
...,...,...,...,...,...
2931534,1000,-0.257,0.514,-11.328,16570.5
2931535,1000,-0.257,0.514,-11.328,16570.7
2931536,1000,-0.257,0.514,-11.328,16570.8
2931537,1000,-0.257,0.515,-11.329,16571.3


In [79]:

# Here we subtract off the first time bin to shift all the EL values to zero

def GetSubDf(data, var, var_sub, query):
    data_copy = data.copy(deep=True)

    data_sub = data_copy.drop_duplicates(subset= ["event"], keep='first')
    data_sub = data_sub.drop(columns = query)

    data_copy['Count'] = 1
    data_copy = data_copy.groupby(["event"]).Count.count().reset_index()

    # get a dataframe with the first time row to subtract from
    data_merge = pd.DataFrame()
    data_sub = data_sub.merge(data_copy, on='event', how="inner")
    data_sub = data_sub.reset_index(drop = True)
    data_sub = data_sub.loc[data_sub.index.repeat(data_sub.Count)]
    data_sub = data_sub.drop(columns = ["Count"])
    data_sub = data_sub.rename(columns={var: var_sub})
    data_sub = data_sub.reset_index(drop=True)
    return data_sub



In [80]:

data_sub = pd.DataFrame()
query = ["x", "y", "z"]
data_sub = GetSubDf(data, "t", "t_sub", query)
data["t"] = data["t"] - data_sub["t_sub"]

data_sub = pd.DataFrame()
query = ["t", "y", "z"]
data_sub = GetSubDf(data, "x", "x_sub", query)
data["x"] = data["x"] - data_sub["x_sub"]

data_sub = pd.DataFrame()
query = ["x", "t", "z"]
data_sub = GetSubDf(data, "y", "y_sub", query)
data["y"] = data["y"] - data_sub["y_sub"]

data_sub = pd.DataFrame()
query = ["x", "y", "t"]
data_sub = GetSubDf(data, "z", "z_sub", query)
data["z"] = data["z"] - data_sub["z_sub"]

display(data)


,event,x,y,z,t
0,1,0.000,0.000,0.000,0.0
1,1,-0.001,-0.002,-0.021,105.0
2,1,0.002,-0.001,-0.022,108.1
3,1,0.003,-0.003,-0.024,113.9
4,1,0.001,-0.003,-0.029,136.3
...,...,...,...,...,...
2931534,1000,-0.015,0.077,-0.687,2346.8
2931535,1000,-0.015,0.077,-0.687,2347.0
2931536,1000,-0.015,0.077,-0.687,2347.1
2931537,1000,-0.015,0.078,-0.688,2347.6


In [81]:
# Export the dataframe 
data = data[ data["event"] < 5000]

# Convert some columns to mm to save space
data.to_csv("../Files/CRAB/CRAB_Profiles.csv", index=False, header=False, float_format='%4.2f')